# Zadanie 5
Miłosz Sadziński 259139

Z racji, że wykonanie oryginalnego ćwiczenia nie było możliwe głównie z powodów kompatybilności pythona postanowiłem przygotować własny model i go przetestować:

In [9]:
import tensorflow as tf

batch_size = 32
img_size = (256, 256)

train_ds = tf.keras.utils.image_dataset_from_directory(
    "Images",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "Images",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

class_names = train_ds.class_names
print("Klasy:", class_names)

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

epochs = 50
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

model.save("model_obrazy_9.keras")


Found 935 files belonging to 5 classes.
Using 748 files for training.
Found 935 files belonging to 5 classes.
Using 187 files for validation.
Klasy: ['biurka', 'drabina', 'drzwi', 'lampy', 'szafy']
Epoch 1/50


C:\Users\iksni\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 474ms/step - accuracy: 0.2408 - loss: 1.6938 - val_accuracy: 0.2460 - val_loss: 1.4691
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 426ms/step - accuracy: 0.3817 - loss: 1.3656 - val_accuracy: 0.5241 - val_loss: 1.2137
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 445ms/step - accuracy: 0.6158 - loss: 1.0759 - val_accuracy: 0.5989 - val_loss: 1.1905
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 494ms/step - accuracy: 0.6935 - loss: 0.8449 - val_accuracy: 0.5882 - val_loss: 1.1297
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 474ms/step - accuracy: 0.8188 - loss: 0.4971 - val_accuracy: 0.5829 - val_loss: 1.3677
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 487ms/step - accuracy: 0.8676 - loss: 0.3698 - val_accuracy: 0.6257 - val_loss: 1.3319
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 480ms/step - accuracy: 0.9617 - loss: 0.1514 - val_accuracy: 0.5829 - val_loss: 1.6033
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 472ms/step - accuracy: 0.9750 - loss: 0.0910 - val_accuracy: 0.604

Po wygenerowaniu modelu został on przetestowany poniższym skryptem, sprawdza on każde zdjęcie i porównuje przewidywaną klasę z tą prawdziwą a na koniec wyświetla statystyki:

In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import tensorflow as tf
from collections import defaultdict

# Parametry
img_size = (256, 256)
folder = "Images"  # folder ze zdjęciami do sprawdzenia
model_path = "model_obrazy_9.keras" #model_obrazy_1.keras to najlepszy model

# Wczytaj model
model = load_model(model_path)

# Pobierz class_names z katalogu treningowego
ds = tf.keras.utils.image_dataset_from_directory(
    "Images",
    image_size=(256, 256),
    batch_size=1
)
class_names = ds.class_names

# Statystyki
stats = defaultdict(lambda: {"correct": 0, "total": 0})

# Funkcja predykcji
def predict_image_class(image_path):
    img = Image.open(image_path).convert("RGB").resize(img_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, 0)
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions)
    return class_names[predicted_class_idx]

# Przetwarzanie testowych obrazów
for root, _, files in os.walk(folder):
    for file in files:
        if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
            image_path = os.path.join(root, file)

            # Odczytaj prawdziwą klasę z folderu nadrzędnego
            true_class = os.path.basename(os.path.dirname(image_path))

            try:
                predicted_class = predict_image_class(image_path)

                stats[true_class]["total"] += 1
                if predicted_class == true_class:
                    stats[true_class]["correct"] += 1

                print(f"{image_path} => przewidziano: {predicted_class}, rzeczywista: {true_class}")
            except Exception as e:
                print(f"Błąd przy {image_path}: {e}")

# Wyniki końcowe
print("\n--- Podsumowanie skuteczności ---")
for class_name, values in stats.items():
    correct = values["correct"]
    total = values["total"]
    accuracy = 100.0 * correct / total if total > 0 else 0.0
    print(f"{class_name}: {correct}/{total} poprawnych ({accuracy:.2f}%)")


Found 935 files belonging to 5 classes.
Images\biurka\000001.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000002.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000003.jpg => przewidziano: lampy, rzeczywista: biurka
Images\biurka\000004.jpg => przewidziano: lampy, rzeczywista: biurka
Images\biurka\000005.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000006.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000007.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000008.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000009.jpg => przewidziano: drzwi, rzeczywista: biurka
Images\biurka\000010.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000011.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000012.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000013.jpg => przewidziano: biurka, rzeczywista: biurka
Images\biurka\000014.jpg => przewidziano: biurka, rze

Jak można zauważyć, model radzi sobie całkiem dobrze, pomijając krzesełka, których nie potrafił wcale zidentyfikować. Możliwą przyczyną tego stanu rzeczy może być niezbyt duża liczba próbek.